In [52]:
import pandas as pd
import numpy as np
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials

# Lê a planiha Conformidade Legal 
#(https://docs.google.com/spreadsheets/d/1_teMusgzqisvbbL3TOONcjJSBibTTae5AIKp-oeceQg/edit?gid=0#gid=0)

# Em resumo, esse trecho do código, lê todas as abas da planilha do google sheets Conformidade Legal
# e monta um dataframe único com todos os dados.
# Esse processo é interessante de ser realizado via Phyton devido a muitas vezes quando da utilização 
# da solução nativa do Power BI (conector do google sheets) termos experimentados erros com relação 
# à quantidade de requisições (Erro: Too many requests).


# Caminho para o arquivo de credenciais
SERVICE_ACCOUNT_FILE = "C:\\Users\\enioa\\BI-CONFORMIDADE\\iconic-apricot-248419-55a189fad524.json"
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
SHEET_ID = "1_teMusgzqisvbbL3TOONcjJSBibTTae5AIKp-oeceQg"

# Autenticação com a API do Google Sheets
credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('sheets', 'v4', credentials=credentials)
sheet = service.spreadsheets()

# Obter todas as abas da planilha
spreadsheet = sheet.get(spreadsheetId=SHEET_ID).execute()
sheet_names = [sheet['properties']['title'] for sheet in spreadsheet['sheets']]

# DataFrame vazio para armazenar todos os links
dados = pd.DataFrame()

# Iterar por cada aba e extrair os dados, ignorando a aba "Dados"
for sheet_name in sheet_names:
    if sheet_name == "Dados":
        continue  # Ignora a aba "Dados"
    
    RANGE = f"{sheet_name}!A1:R1000"  # Ajuste o intervalo conforme necessário
    result = sheet.values().get(spreadsheetId=SHEET_ID, range=RANGE, valueRenderOption='FORMATTED_VALUE').execute()
    values = result.get('values', [])

    if values:
        # Verificar se o número de colunas no cabeçalho é igual ao número de colunas nos dados
        num_columns = len(values[0])
        for row in values[1:]:
            while len(row) < num_columns:
                row.append("")  # Preencher colunas vazias com string vazia
            while len(row) > num_columns:
                row.pop()  # Remover colunas extras

        # Criar DataFrame a partir da aba atual
        dados_aux = pd.DataFrame(values[1:], columns=values[0])  # Primeira linha como cabeçalho
        dados=pd.concat([dados,dados_aux],ignore_index=True)
        # Verificar os nomes das colunas
        #print(f"Colunas da aba '{sheet_name}': {df.columns.tolist()}")

dados["IAD"]= dados["ESTADO DE CUMPRIMENTO"].apply(
                lambda x:"" if x in ["Em Análise","Em análise","Não se aplica"] else x)
dados['NORMA - ORIGEM']=dados['NORMA']+" - "+dados['ORIGEM']                
        
# Salvar os links em um arquivo CSV
dados.to_csv('C:\\Users\\enioa\\BI-CONFORMIDADE\\Dados.csv', index=False)
print("Arquivo 'dados.csv' foi gerado com sucesso.")

# A partir desse ponto inicia-se a geração da tabela de links. A partir dela é que vamos identificar
# quais são as normas que já têm o questionário respondido a fim de que se possa incluir as informações
# no BI Conformidade.
# O resultado é incluído na variável "links_filtrados" e salvo em um arquivo .csv de mesmo nome.

links=dados[["NORMA","ORIGEM","LINK PARA FORMULÁRIO","ÁREA RESPONSÁVEL"]]
# Remover espaços extras das células da coluna 'LINK PARA FORMULÁRIO'
links['LINK PARA FORMULÁRIO'] = links['LINK PARA FORMULÁRIO'].str.strip()
links['NORMA - ORIGEM']=links['NORMA']+" - "+links['ORIGEM']

# Filtrar o DataFrame para manter apenas as linhas com link informado (não vazio ou nulo)
links_filtrados = links[links['LINK PARA FORMULÁRIO'].str.startswith('https://', na=False)]

# Opcionalmente, verificar se o filtro foi aplicado corretamente:
print(f"Total de linhas antes do filtro: {len(links)}")
print(f"Total de linhas após o filtro: {len(links_filtrados)}")

# Salvar os links filtrados em um novo arquivo CSV
links_filtrados.to_csv('links_filtrados_.csv', index=False)
print("Arquivo 'links_filtrados.csv' foi gerado com sucesso.")
#print(links_filtrados)



Arquivo 'dados.csv' foi gerado com sucesso.
Total de linhas antes do filtro: 998
Total de linhas após o filtro: 113
Arquivo 'links_filtrados.csv' foi gerado com sucesso.


C:\Users\enioa\AppData\Local\Temp\ipykernel_28896\3155501137.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  links['LINK PARA FORMULÁRIO'] = links['LINK PARA FORMULÁRIO'].str.strip()
C:\Users\enioa\AppData\Local\Temp\ipykernel_28896\3155501137.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  links['NORMA - ORIGEM']=links['NORMA']+" - "+links['ORIGEM']


In [59]:
# Nesse trecho, geramos uma base única com todas as informações provenientes de todas as planilhas
# dos questionários respondidos. O resultado é salvo na variável "base_transposta" e em um arquivo
# .csv denomeminado "base.csv". 


# Ler arquivo com links
links_filtrados = pd.read_csv(r'C:\Users\enioa\BI-CONFORMIDADE\links_filtrados.csv')

# Extrair IDs das planilhas
links_filtrados['LINK PLANILHA'] = links_filtrados['LINK PARA FORMULÁRIO'].str.extract(r'/d/([^/]+)/')
SHEET_IDS_ITENS = links_filtrados['LINK PLANILHA'].dropna().tolist()

# Inicializar lista para armazenar os DataFrames temporários
dataframes = []
base_transposta=pd.DataFrame()
# Processar cada planilha
for index, sheet_id_item in enumerate(SHEET_IDS_ITENS):
    try: 
        # Obter valores da planilha
        RANGE_ITEM = "A1:ZZZ2"
        resultado = sheet.values().get(spreadsheetId=sheet_id_item, range=RANGE_ITEM, valueRenderOption='FORMATTED_VALUE').execute()
        valores = resultado.get('values', [])
        
        if valores:
            # Ajustar inconsistências no número de colunas
            num_columns = len(valores[0])
            valores = [row + [""] * (num_columns - len(row)) for row in valores]
            
            # Criar DataFrame com os dados da planilha
            base_aux = pd.DataFrame(valores[1:], columns=valores[0])
            
            # Adicionar a linha 'norma' ao final do DataFrame temporário
            norma_row = [links_filtrados.iloc[index]['NORMA - ORIGEM']] * len(valores[0])  # Preencher a linha com a norma
            origem_row = [links_filtrados.iloc[index]['ORIGEM']] * len(valores[0])  # Preencher a linha com a origem
            area_row= [links_filtrados.iloc[index]['ÁREA RESPONSÁVEL']] * len(valores[0])  # Preencher a linha com a área responsável
            base_aux.loc[len(base_aux)] = norma_row  # Adiciona a linha ao final do DataFrame
            base_aux.loc[len(base_aux)] = origem_row  # Adiciona a linha ao final do DataFrame
            base_aux.loc[len(base_aux)] = area_row  # Adiciona a linha ao final do DataFrame
            
            # Adicionar o DataFrame à lista
            #dataframes.append(base_aux)
            #base = pd.concat(dataframes, ignore_index=True)
            base_transposta_aux = base_aux.transpose()
            base_transposta=pd.concat([base_transposta,base_transposta_aux],ignore_index=False)
            
    except Exception as e:
        print(f"Erro ao processar a planilha {sheet_id_item}: {e}")

base_transposta = base_transposta.drop(['Carimbo de data/hora','Comentário / Evidências'], axis=0)


# Concatenar todos os DataFrames
#base = pd.concat(dataframes, ignore_index=True)

# Transpor o DataFrame
#base_transposta = base.transpose()
#base_transposta.reset_index(inplace=True)

# Salvar o DataFrame transposto como CSV
output_path = r'C:\Users\enioa\BI-CONFORMIDADE\base.csv'
base_transposta.to_csv(output_path, index=True)

print(f"Arquivo salvo em: {output_path}")


Arquivo salvo em: C:\Users\enioa\BI-CONFORMIDADE\base.csv


In [54]:
# A fim de comparar a eficiência de montar as tabelas dimensão diretamente no power bi e fora dele, 
# vou gerar as tabelas dimensão em arquivos .csv com base na variável "dados" que está no primeiro 
# chunck desse código.

origem = pd.DataFrame(dados['ORIGEM'].unique())
tema = pd.DataFrame(dados['TEMA'].unique())
tipo_norma = pd.DataFrame(["Resolução"])
area= pd.DataFrame(dados['ÁREA RESPONSÁVEL'].unique())
estado_analise= pd.DataFrame(['Analisado','Em Análise','Não se Aplica'])
ano= pd.DataFrame(dados['DATA DE REGISTRO'].str.extract(r'(\d{4}$)',expand=False).unique())
situacao= pd.DataFrame(dados['SITUAÇÃO'].unique())
norma=pd.DataFrame(dados['NORMA - ORIGEM'].unique())

origem.to_csv(r'C:\Users\enioa\BI-CONFORMIDADE\tab_dimensao\dOrigem.csv')
tema.to_csv(r'C:\Users\enioa\BI-CONFORMIDADE\tab_dimensao\dTema.csv')
tipo_norma.to_csv(r'C:\Users\enioa\BI-CONFORMIDADE\tab_dimensao\dTipo_de_Norma.csv')
area.to_csv(r'C:\Users\enioa\BI-CONFORMIDADE\tab_dimensao\dArea.csv')
estado_analise.to_csv(r'C:\Users\enioa\BI-CONFORMIDADE\tab_dimensao\dEstado_de_Analise.csv')
ano.to_csv(r'C:\Users\enioa\BI-CONFORMIDADE\tab_dimensao\dAno.csv')
situacao.to_csv(r'C:\Users\enioa\BI-CONFORMIDADE\tab_dimensao\dSituacao.csv')
norma.to_csv(r'C:\Users\enioa\BI-CONFORMIDADE\tab_dimensao\dNorma.csv')

In [29]:
a=pd.DataFrame(base_transposta.iloc[:,[1]])

In [58]:
base_transposta.head()

,0,1,2
"Art. 1º É instituído o Sistema de Gestão Estratégica da Justiça do Trabalho – SIGEST no âmbito da Justiça do Trabalho de 1º e 2º graus, com vistas a auxiliar a implantação e a gestão da estratégia do Conselho Superior da Justiça do Trabalho e dos Tribunais Regionais do Trabalho. \n§1° O SIGEST consiste em ferramenta tecnológica para a medição, monitoramento e análise das estratégias definidas pelo Conselho Superior da Justiça do Trabalho e pelos Tribunais Regionais do Trabalho.\n§ 2° O acompanhamento da execução da gestão estratégica do Conselho Superior da Justiça do Trabalho e dos Tribunais Regionais do Trabalho será realizada exclusivamente por meio do SIGEST, nos termos desta Resolução. \n\nNão se infere deste artigo comando para providências da área, portanto não se aplica para item de conformidade.\n",Concordo,Resolução nº 122/2013 - CSJT,CSJT
"Art. 2° A utilização do SIGEST pelo CSJT e pelos órgãos da Justiça do Trabalho de 1° e 2° graus objetiva:\nI – proporcionar o alinhamento estratégico da Justiça do Trabalho mediante o acompanhamento dos indicadores estratégicos do Conselho Superior da Justiça do Trabalho e dos Tribunais Regionais do Trabalho, visando à melhoria dos processos de trabalho e à superação dos desafios descritos na estratégia;\nII – propiciar a gestão da execução do plano estratégico institucional e do plano estratégico de tecnologia da informação – PETI-JT;\nIII – dotar o Conselho Superior da Justiça do Trabalho e os Tribunais Regionais do Trabalho de infraestrutura tecnológica de suporte à execução dos respectivos planos estratégicos;\nIV - facilitar o entendimento e a realização do desdobramento da estratégia, em perspectivas, temas, objetivos, indicadores e metas;\nV – assegurar a transparência da gestão pública do Conselho Superior da Justiça do Trabalho e dos Tribunais Regionais do Trabalho.\n\nNão se infere deste artigo comando para providências da área, portanto não se aplica para item de conformidade.\n",Concordo,Resolução nº 122/2013 - CSJT,CSJT
"Art. 3º O Conselho Superior da Justiça do Trabalho e os Tribunais Regionais do Trabalho deverão inserir no SIGEST os seguintes dados:\nI – os indicadores nacionais e os indicadores da Justiça do Trabalho;\nII – os índices de cumprimento das metas nacionais;\nIII – os indicadores e as metas específicos do CSJT e de cada TRT;\nIV – os planos de ação/projetos do Conselho Superior da Justiça do Trabalho e dos Tribunais Regionais do Trabalho.\n§ 1° Em caso de inviabilidade de inserção de todos os indicadores previstos no inciso I, deverão ser lançados no SIGEST, no mínimo, os dados previstos nos anexos desta Resolução.\n§ 2° Os planos táticos e operacionais das unidades judiciárias e administrativas dos Tribunais Regionais do Trabalho, a critério das respectivas Presidências, poderão ser inseridos no SIGEST.\n\nO TRT-7 inseriu no SIGEST os seguintes dados:\nI – os indicadores nacionais e os indicadores da Justiça do Trabalho;\nII – os índices de cumprimento das metas nacionais;\nIII – os indicadores e as metas específicos do CSJT e do TRT-7?",Sim,Resolução nº 122/2013 - CSJT,CSJT
"Art. 4º As sugestões dos Tribunais Regionais do Trabalho de melhorias no SIGEST deverão ser encaminhadas ao Conselho Superior da Justiça do Trabalho, a quem compete analisar as propostas e, caso aprovadas, implementar novas funcionalidades, mantendo a padronização do sistema.\n\nNão se infere deste artigo comando para providências da área, portanto não se aplica para item de conformidade.",Concordo,Resolução nº 122/2013 - CSJT,CSJT
"Art. 5º O Conselho Superior da Justiça do Trabalho atuará como órgão gestor do SIGEST, competindo:\nI - à Coordenadoria de Gestão Estratégica gerenciar o planejamento estratégico do Conselho Superior da Justiça do Trabalho e da Justiça do Trabalho de 1° e 2° graus;\nII - à Coordenadoria de Tecnologia da Informação e Comunicação gerenciar o Plano Estratégico de Tecnologia da Informação da Justiça do Trabalho – PETI-JT.\n§ 1º As áreas de planeja